In [1]:
import collections
import math
import os
import random
import zipfile
import numpy as np
import urllib
import tensorflow as tf

/usr/local/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## cannot download

In [8]:
url = 'http://mattmahoney.net/dc/'
def maybe_download(filename, expected_bytes):
    if not os.path.exists(filename):
        filename, _ = urllib.request.urlretrieve(url + filename, filename)
    statinfo = os.stat(filename)
    print(statinfo)
    if statinfo.st_size == expected_bytes:
        print("Found and verified", filename)
    else:
        print(statinfo.st_size)
        raise Exception('Failed to verify ' + filename + '. Can you get to it with a browser?')
    return filename
filename = maybe_download('text8.zip', 31344016)

os.stat_result(st_mode=33188, st_ino=8593844934, st_dev=16777220, st_nlink=1, st_uid=501, st_gid=20, st_size=278888, st_atime=1526306298, st_mtime=1526306293, st_ctime=1526306293)
Found and verified text8.zip


In [2]:
def read_data(filename):
    with zipfile.ZipFile(filename) as f:
        data = tf.compat.as_str(f.read(f.namelist()[0])).split()
    return data
words = read_data('./text8.zip')
print("Data size", len(words))

Data size 17005207


In [3]:
vocabulary_size = 50000
def build_dataset(words):
    count = [['UNK', -1]]
    count.extend(collections.Counter(words).most_common(vocabulary_size -1))
    dictionary = dict()
    for word, _ in count:
        # 單字編號
        dictionary[word] = len(dictionary)
    data = list()
    unk_count = 0
    for word in words:
        if word in dictionary:
            index = dictionary[word]
        else:
            index = 0
            unk_count += 1
        data.append(index)
    count[0][1] = unk_count
    reverse_dictionary = dict(zip(dictionary.values(), dictionary.keys()))
    return data, count, dictionary, reverse_dictionary

# data: tokenize raw data
# count: word frequency(top 50000)
# dictionary: word and token map
# reverse_dictionary: reversed dictionary
data, count, dictionary, reverse_dictionary = build_dataset(words)

In [4]:
del words
print('Most common words +(UNK)', count[:5])
print('Sample data', data[:10], [reverse_dictionary[i] for i in data[:10]])

Most common words +(UNK) [['UNK', 418391], ('the', 1061396), ('of', 593677), ('and', 416629), ('one', 411764)]
Sample data [5234, 3081, 12, 6, 195, 2, 3134, 46, 59, 156] ['anarchism', 'originated', 'as', 'a', 'term', 'of', 'abuse', 'first', 'used', 'against']


In [5]:
data_index = 0
# batch_size = 一個批次的sample個數
# num_skips = 從鄰居中選幾個樣本
# skip_window = 幾個鄰居
def generate_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape = (batch_size), dtype = np.int32)
    labels = np.ndarray(shape = (batch_size, 1), dtype = np.int32)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen = span)
    # 從頭(data_index = 0)開始把單字一個一個塞進buffer內
    # 這個迴圈只跑一次，因為之後每從右邊加單字，左邊會自動被擠掉，維持max_len = span
    for _ in range(span):
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    # print(buffer)
    # 每跑一圈就完成 num_skips 個樣本，共要跑 batch_size//num_skips 圈
    for i in range(batch_size // num_skips):
        target = skip_window
        # 第skip_window個單字是目標，要找的是他的鄰居而不是他自己，所以加入targets_to_avoid
        # targets_to_avoid 是在buffer裏被禁用的單字index
        targets_to_avoid = [skip_window]
        for j in range(num_skips):
            while target in targets_to_avoid:
                # 個人註記：其實可以不用亂數產生，因為已經限制batch_size是num_skips的整數倍，所以每個單字都會找滿所有鄰居
                # 照順序加入後在打亂順序應該也可以，不需要亂數一直while loop
                target = random.randint(0, span - 1)
            targets_to_avoid.append(target)
            batch[i * num_skips + j] = buffer[skip_window]
            # labels是2維陣列，所以多一個0
            labels[i * num_skips + j, 0] = buffer[target]
        buffer.append(data[data_index])
        data_index = (data_index + 1) % len(data)
    return batch, labels

In [6]:
batch, labels = generate_batch(8, 2, 1)
for i in range(8):
    print(batch[i], reverse_dictionary[batch[i]], 
          '->', labels[i, 0], reverse_dictionary[labels[i, 0]])

3081 originated -> 12 as
3081 originated -> 5234 anarchism
12 as -> 6 a
12 as -> 3081 originated
6 a -> 12 as
6 a -> 195 term
195 term -> 2 of
195 term -> 6 a


In [21]:
batch_size = 128
embedding_size = 128    # 可以視為encoded後的向量維度
skip_window = 1
num_skips = 2

valid_size = 16
valid_window = 100      # top 100 的高頻單字
valid_example = np.random.choice(valid_window, valid_size, replace=True)
num_sampled = 64


In [23]:
graph = tf.Graph()
with graph.as_default():
    train_inputs = tf.placeholder(tf.int32, shape=[batch_size])
    train_label = tf.placeholder(tf.int32, shape=[batch_size, 1])
    valid_dataset = tf.constant(valid_example, dtype=tf.int32)
    
    with tf.device('/cpu:0'):
        # tf 矩陣維度要倒過來看(行，列)
        # embeddings 是一個128 * 50000 的矩陣
        # inputs 是一個50000的向量
        embeddings = tf.Variable(tf.random_uniform([vocabulary_size, embedding_size], -1.0, 1.0))
        # inputs is sparse vecter, all we need to find is which column value is 1 -> embed
        embed = tf.nn.embedding_lookup(embeddings, train_inputs)
        
        nce_weights = tf.Variable(tf.truncated_normal([vocabulary_size, embedding_size], stddev=1.0 / math.sqrt(embedding_size)))
        nce_biases = tf.Variable(tf.zeros([vocabulary_size]))
    loss = tf.reduce_mean(tf.nn.nce_loss(weights=nce_weights,
                                        biases=nce_biases,
                                        labels = train_label,
                                        inputs = embed,
                                        num_sampled = num_sampled,
                                        num_classes = vocabulary_size))
    optimizier = tf.train.GradientDescentOptimizer(1.0).minimize(loss)
    norm = tf.sqrt(tf.reduce_sum(tf.square(embeddings), 1, keepdims=True))
    normalized_embeddings = embeddings / norm
    valid_embeddings = tf.nn.embedding_lookup(normalized_embeddings, valid_dataset)
    similarity = tf.matmul(valid_embeddings, normalized_embeddings, transpose_b=True)
    
    init = tf.global_variables_initializer()

In [27]:
num_step = 100001

with tf.Session(graph=graph) as session:
    init.run()
    print("Initialized")
    
    average_loss = 0
    for step in range(num_step):
        batch_inputs, batch_labels = generate_batch(batch_size, num_skips, skip_window)
        feed_dict = {train_inputs: batch_inputs, train_label: batch_labels}
        _, loss_val = session.run([optimizier, loss], feed_dict = feed_dict)
        average_loss += loss_val
        if step % 2000 ==0:
            if step > 0:
                average_loss/=2000
            print("Average loss at step ", step, ": ", average_loss)
            average_loss = 0
        if step % 10000 == 0:
            sim = similarity.eval()
            for i in range(valid_size):
                valid_word = reverse_dictionary[valid_example[i]]
                top_k = 8
                nearest = (-sim[i, :]).argsort()[1: top_k +1]
                log_str = "Nearest to %s:" % valid_word
                for k in range(top_k):
                    close_word = reverse_dictionary[nearest[k]]
                    log_str = "%s %s," %(log_str, close_word)
                print(log_str)
    final_embeddings = normalized_embeddings.eval()

Initialized
Average loss at step  0 :  275.9525146484375
Nearest to all: unrepentant, solidifies, bacon, predating, fossilized, foucault, humor, airstrips,
Nearest to more: trakai, nonsignatory, paraguayan, moorish, richelieu, usp, rohe, rented,
Nearest to will: glyn, pastors, innovator, dae, poster, dez, occurring, colonizing,
Nearest to they: unless, boleslav, sensibilities, tavistock, infective, freie, participatory, mammoths,
Nearest to there: sober, nas, tabby, reverting, enclosure, veterinarians, perpendicularly, palpatine,
Nearest to use: granting, bruised, jumaada, motivations, collingwood, service, leftover, amo,
Nearest to most: inline, steal, georgi, automakers, ce, ontario, mclachlan, flirting,
Nearest to known: unified, evergreen, southeastern, weeping, shoot, chaosium, exploited, masse,
Nearest to however: towards, tyr, theories, macgregor, kassel, validating, tinkering, antiquarian,
Nearest to united: ministers, ssr, groves, ceiling, enrichment, facilities, intervene, zi

Average loss at step  52000 :  5.167786599516869
Average loss at step  54000 :  5.128866054296494
Average loss at step  56000 :  5.066076731562615
Average loss at step  58000 :  5.099835057854652
Average loss at step  60000 :  4.944652784943581
Nearest to all: many, these, microcebus, wct, both, three, akita, assurance,
Nearest to more: less, provable, piece, hellfire, emigrant, reuptake, very, most,
Nearest to will: would, can, may, could, must, to, pastors, paradoxes,
Nearest to they: he, there, we, it, you, not, who, these,
Nearest to there: it, they, now, chanter, burundi, which, this, hbox,
Nearest to use: amo, granting, microcebus, michelob, collingwood, thibetanus, service, acropolis,
Nearest to most: trinomial, akita, nuke, fao, ce, polyhedra, designation, agave,
Nearest to known: used, unified, vicinity, well, defenders, such, tactical, southeastern,
Nearest to however: but, tamarin, where, reginae, microcebus, abet, that, wct,
Nearest to united: agouti, microsite, ssbn, imran